# 0. Configuración inicial:

In [55]:
from uuid import uuid4

import fitz
import tiktoken
from langchain_chroma import Chroma
from langchain_core.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [56]:
from naive_rag import config

In [57]:
OPENAI_EMBEDDING_PRICE_PER_1_M = 0.02

OPENAI_GPT_4O_MINI_INPUT_PER_1M = 0.150
OPENAI_GPT_4O_MINI_OUTPUT_PER_1M = 0.6

# 1. Cargamos los datos no estructurados:

Vamos a utilizar la constitución chilena como ejemplo:

In [58]:
def read_pdf_with_fitz(pdf_path):
    """Read text from a PDF file using fitz."""
    try:
        text = ''
        with fitz.open(pdf_path) as pdf:
            for page_num in range(len(pdf)):
                page = pdf[page_num]
                text += page.get_text() + '\n'
        return text.strip()
    except Exception as e:
        return f'An error occurred: {e}'

In [59]:
constitucion_texto = read_pdf_with_fitz(config.DIR_PATH / 'data' / 'constitucion.pdf')

Veamos un extracto:

In [60]:
print(constitucion_texto[3000:3750])

 permitan a 
todos y a cada uno de los integrantes de la comunidad nacional su mayor realización 
espiritual y material posible, con pleno respeto a los derechos y garantías que esta 
Constitución establece.  
 
Es deber del Estado resguardar la seguridad nacional, dar protección a la 
población y a la familia, propender al fortalecimiento de ésta, promover la integración 
armónica de todos los sectores de la Nación y asegurar el derecho de las personas a 
participar con igualdad de oportunidades en la vida nacional.  
 
 
 
 
1 Texto actualizado al 1 de octubre de 2005. Incluye las reformas introducidas por las leyes 18.825, 19.055, 19.097, 19.295, 
19.448 , 19.519, 19.526 , 19.541, 19.597, 19.611, 19.634, 19.643 , 19.671, 19.672, 19.742 ;


# 2. Dividimos los documentos en Chunks:

Definimos un text splitter que va a dividir cada dato en *chunks* de texto.

In [61]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64,
    length_function=len,
    is_separator_regex=True,  # defaulting to ["\n\n", "\n", " ", ""]
)

In [62]:
documents = text_splitter.create_documents([constitucion_texto])

Se generan documentos, que poseen dos atributos: **metadata** y **page_content**.

In [63]:
documents[0]

Document(metadata={}, page_content='CONSTITUCIÓN POLÍTICA DE LA REPÚBLICA \n \n \n \n \n \n \n \n \n \n \n \nTexto actualizado a octubre  de 2010 \n \n \n \n \n \n\n \n \n2')

In [64]:
print(f'Generamos {len(documents)} documentos')

Generamos 337 documentos


# 3. Estimación de costo:

Un paso importante siempre es estimar el costo:

In [65]:
def count_tokens(text: str, model: str) -> int:
    """Count the number of tokens in a text string for the OpenAI model."""
    encoding = tiktoken.encoding_for_model(model)

    return len(encoding.encode(text))

In [66]:
tokens_documentos = sum([count_tokens(doc.page_content, 'text-embedding-3-small') for doc in documents])

In [67]:
(tokens_documentos / 1_000_000) * OPENAI_EMBEDDING_PRICE_PER_1_M * 40

0.06501040000000001

In [68]:
ejemplo_input_string = '¿Qué derechos protege la Constitución?'

In [69]:
ejemplo_output_string = 'Artículo 19. La Constitución asegura a todas las personas: 1°. El derecho a la vida y a la integridad física y psíquica de la persona'  # noqa: E501

In [70]:
precio_input = (count_tokens(ejemplo_input_string, 'gpt-4o-mini') / 1_000_000) * OPENAI_GPT_4O_MINI_INPUT_PER_1M
precio_output = (count_tokens(ejemplo_output_string, 'gpt-4o-mini') / 1_000_000) * OPENAI_GPT_4O_MINI_OUTPUT_PER_1M

In [71]:
(precio_input + precio_output) * 10 * 40

0.008280000000000001

# 4. Ingestamos los documentos en la vector store:

In [72]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small', api_key=config.OPENAI_API_KEY)

In [73]:
vector_store = Chroma(
    collection_name='legal-data',
    embedding_function=embedding_model,
)

In [74]:
uuids = [str(uuid4()) for _ in range(len(documents))]

_ = vector_store.add_documents(documents=documents, ids=uuids)

Definimos una función para recopilar documentos según similitud:

In [75]:
def retrieve(question: str) -> str:
    """Retrieve the top 3 most similar documents to a question."""
    retrieved_docs = vector_store.similarity_search(question, k=3)
    return ';'.join([doc.page_content for doc in retrieved_docs])

# 5. Instanciamos el modelo de lenguaje:

Creamos un prompt para nuestro RAG:

In [76]:
template = """
Eres un asistente que ayuda a los estudiantes de derecho a encontrar información en la Constitución de Chile.

La pregunta es: {question}

Debes responder solo utilizando el contenido del siguiente contexto entregado:

{context}

Tu respuesta debe ser respondida como un académico del derecho. Debes ser precisa tu respuesta, no te extiendas innecesariamente.

Entrégame la fuente de tu respuesta sin modificar el extracto del contexto entregado para que pueda verificar la información.
"""  # noqa: E501

In [77]:
prompt = PromptTemplate(input_variables=['question', 'context'], template=template)

Instanciamos el modelo de OpenAI:

In [78]:
llm = ChatOpenAI(model='gpt-4o-mini', api_key=config.OPENAI_API_KEY, temperature=0.2)

Creamos una función para generar respuestas:

In [79]:
def generate(question: str, context) -> str:
    """Generate a response to a question given a context."""
    messages = prompt.invoke({'question': question, 'context': context})
    return llm.invoke(messages).content

# 6. ¡Juega con el modelo!

In [80]:
pregunta = '¿Es necesario ser chileno para ser Presidente de la República?'

In [81]:
context = retrieve(pregunta)
print(generate(pregunta, context))

Sí, es necesario ser chileno para ser Presidente de la República. Según el Artículo 25 de la Constitución de Chile, para ser elegido Presidente se requiere tener la nacionalidad chilena, además de cumplir con otros requisitos como tener al menos treinta y cinco años de edad y poseer las calidades necesarias para ser ciudadano con derecho a sufragio.

Fuente: Artículo 25. Para ser elegido Presidente de la República se requiere tener la nacionalidad chilena de acuerdo a lo dispuesto en los números 1º ó 2º del artículo 10; tener cumplidos treinta y cinco años de edad y poseer las demás calidades necesarias para ser ciudadano con derecho a sufragio.


In [83]:
pregunta = '¿Cuál es el deber fundamental como chileno?'

In [84]:
context = retrieve(pregunta)
print(generate(pregunta, context))

El deber fundamental como chileno es "honrar a la patria, de defender su soberanía y de contribuir a preservar la seguridad nacional y los valores esenciales de la tradición chilena". Además, se establece que "el servicio militar y demás cargas personales que imponga la ley son obligatorios en los términos y formas que ésta determine".

Fuente: Artículo 22.
